# Convolutional Neural Network

### Importing the libraries

In [48]:
#Magic code to install programs in jupyter
import sys
!{sys.executable} -m pip install tensorflow
!{sys.executable} -m pip install keras
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [49]:
tf.__version__

'2.2.0'

## Part 1 - Data Preprocessing

### Generating images for the Training set

In [50]:
#apply transformations on training set only, needed to advoid overfitting on the model (Geometry, Rotation, Zoom)
#also called immage augmentation, adding new (diverse) immages
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

### Generating images for the Test set

In [51]:
#no transformations to test set, but you do need feature scaling
test_datagen = ImageDataGenerator(rescale = 1./255)

### Creating the Training set

In [52]:
#only thing to modify is the directory to let the algorithem know where to find the training set
#You can change target side if you want, larger target slower processing (size of immage in pixels)
#batch size is a default (usually 32)
#class mode is binary because you have two outcomes, cat or dog
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Creating the Test set

In [53]:
#target and batch size needs to match training set to use the predict method
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [54]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [55]:
#filters = 32 is default value
#kernal_size = 3 means 3x3 box
#activation = recursive function untill output layer
#imput_shape = size of image as identified in data preprocessing, 3 layerss beause the immages are color(RBG)
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, activation = 'relu', input_shape =[64, 64, 3]))

### Step 2 - Pooling

In [56]:
#pool size = 2 is default
#strides = 2 is default, kernal moves across 2 pixels at a time
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

### Adding a second convolutional layer

In [57]:
#repeat convolution and pooling code but omit input shape, as it is only necessary in the first layer of convolution
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, activation = 'relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

### Step 3 - Flattening

In [58]:
#flatten your poolings in to a 1-D vector
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [59]:
#units is the number of hidden neuorons 128 default
#activation is still relu
cnn.add(tf.keras.layers.Dense(units = 128, activation ='relu'))

### Step 5 - Output Layer

In [60]:
#need to modify units to one (binary ast ouput), and activation to sigmoid (as output as binary), softmax for everying else
cnn.add(tf.keras.layers.Dense(units = 1, activation ='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [61]:
#built the same as ANN because the classification is again binary
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [62]:
#x is train set, validation data is test set, epochs is number of rounds testing (can be increased or decreased)
cnn.fit(x = training_set, validation_data= test_set, epochs= 25)

Epoch 1/25
250/250 [==============================] - 30s 118ms/step - loss: 0.6797 - accuracy: 0.5645 - val_loss: 0.6224 - val_accuracy: 0.6485
Epoch 2/25
250/250 [==============================] - 32s 129ms/step - loss: 0.6101 - accuracy: 0.6714 - val_loss: 0.6098 - val_accuracy: 0.6590
Epoch 3/25
250/250 [==============================] - 30s 120ms/step - loss: 0.5666 - accuracy: 0.7089 - val_loss: 0.5654 - val_accuracy: 0.7095
Epoch 4/25
250/250 [==============================] - 30s 120ms/step - loss: 0.5261 - accuracy: 0.7425 - val_loss: 0.5404 - val_accuracy: 0.7260
Epoch 5/25
250/250 [==============================] - 30s 119ms/step - loss: 0.5098 - accuracy: 0.7520 - val_loss: 0.4854 - val_accuracy: 0.7750
Epoch 6/25
250/250 [==============================] - 33s 133ms/step - loss: 0.4897 - accuracy: 0.7642 - val_loss: 0.4802 - val_accuracy: 0.7775
Epoch 7/25
250/250 [==============================] - 29s 114ms/step - loss: 0.4779 - accuracy: 0.7701 - val_loss: 0.4640 - val_ac

### Making a Single Prediction

In [68]:
# two arguments, first is the location of the immage, 2nd is the formatting (image size)
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_11.jpg', target_size = (64, 64))
# now must convert the test image in to an array to use the predict method (PIL - Numpy)
test_image = image.img_to_array(test_image)
#now we have to add fake batch dimension, because our model was trained in batches, we code the fake dimension as 0
test_image = np.expand_dims(test_image, axis = 0)
#now we will code our result variable (0 or 1) binary
result = cnn.predict(test_image)
#code that will tell us what 0 or 1 means
training_set.class_indices
if result[0][0] == 1:
    prediction = 'Dog'
else:
    prediction = 'Cat'
print (prediction)

Cat
